### Preliminares


In [ ]:
from numpy import ndarray
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
from IPython import display
plt.style.use('seaborn-white')
%matplotlib inline

from copy import deepcopy
from collections import deque
from scipy.special import logsumexp

#### Definimos las funciones de activación a utilizar

In [ ]:
def sigmoid(x: ndarray):
    return 1 / (1 + np.exp(-x))


def dsigmoid(x: ndarray):
    return sigmoid(x) * (1 - sigmoid(x))


def tanh(x: ndarray):
    return np.tanh(x)


def dtanh(x: ndarray):
    return 1 - np.tanh(x) * np.tanh(x)


def softmax(x, axis=None):
    return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


def batch_softmax(input_array: ndarray):
    out = []
    for row in input_array:
        out.append(softmax(row, axis=1))
    return np.stack(out)
    

##### Optimizador de redes neuronales recurrentes

In [ ]:
class RNNOptimizer(object):
    # Constructor de la clase optimizador para redes neuronales recurrentes.
    # Args:
    #   lr (float): Tasa de aprendizaje, con un valor por defecto de 0.01.
    #   gradient_clipping (bool): Indica si se aplica clipping a los gradientes,
    #                             activado por defecto.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        self.lr = lr  # Almacena la tasa de aprendizaje.
        self.gradient_clipping = gradient_clipping  # Almacena la configuración de clipping.
        self.first = True  # Variable auxiliar, posiblemente para controlar la primera actualización.

    # Método que ejecuta un paso de optimización sobre todos los parámetros del modelo.
    def step(self) -> None:
        # Itera sobre cada capa del modelo.
        for layer in self.model.layers:
            # Itera sobre cada parámetro de la capa.
            for key in layer.params.keys():
                # Si el clipping de gradientes está activado, aplica clipping.
                if self.gradient_clipping:
                    # Los gradientes se limitan a estar entre -2 y 2 para evitar la explosión de gradientes.
                    np.clip(layer.params[key]['deriv'], -2, 2, layer.params[key]['deriv'])

                # Llama a la regla de actualización para ajustar el valor del parámetro
                # usando la tasa de aprendizaje y el gradiente actual.
                self._update_rule(param=layer.params[key]['value'],
                                  grad=layer.params[key]['deriv'])

    # Método abstracto para definir la regla de actualización de parámetros.
    # Debe ser implementado por subclases.
    def _update_rule(self, **kwargs) -> None:
        raise NotImplementedError("Este método debe ser implementado por subclases.")


#### Algunos optimizadores

In [ ]:
class SGD(RNNOptimizer):
    # Constructor de la clase SGD (Stochastic Gradient Descent).
    # Args:
    #   lr (float): Tasa de aprendizaje.
    #   gradient_clipping (bool): Indica si se activa el clipping de gradientes.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        super().__init__(lr, gradient_clipping)  # Inicializa la clase base

    # Método específico de SGD para actualizar los parámetros según el gradiente.
    def _update_rule(self, **kwargs) -> None:
        update = self.lr * kwargs['grad']  # Calcula la actualización del parámetro.
        kwargs['param'] -= update  # Actualiza el parámetro restando la actualización.
        
class AdaGrad(RNNOptimizer):
    # Constructor de la clase AdaGrad.
    # Args:
    #   lr (float): Tasa de aprendizaje.
    #   gradient_clipping (bool): Indica si se activa el clipping de gradientes.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        super().__init__(lr, gradient_clipping)  # Inicializa la clase base
        self.eps = 1e-7  # Un pequeño número para evitar división por cero en la actualización.

    # Método que ejecuta un paso de optimización sobre todos los parámetros del modelo.
    def step(self) -> None:
        if self.first:
            self.sum_squares = {}  # Diccionario para almacenar la suma acumulada de los cuadrados de los gradientes.
            # Inicializa sum_squares para cada parámetro en cada capa.
            for i, layer in enumerate(self.model.layers):
                self.sum_squares[i] = {}
                for key in layer.params.keys():
                    self.sum_squares[i][key] = np.zeros_like(layer.params[key]['value'])
            self.first = False  # Marca la inicialización como completa.

        for i, layer in enumerate(self.model.layers):
            for key in layer.params.keys():
                if self.gradient_clipping:
                    # Si el clipping de gradientes está activado, aplica clipping.
                    np.clip(layer.params[key]['deriv'], -2, 2, layer.params[key]['deriv'])

                # Actualiza cada parámetro utilizando la regla específica de AdaGrad.
                self._update_rule(param=layer.params[key]['value'],
                                  grad=layer.params[key]['deriv'],
                                  sum_square=self.sum_squares[i][key])

    # Método específico de AdaGrad para actualizar los parámetros.
    def _update_rule(self, **kwargs) -> None:
        # Actualiza la suma acumulada de los cuadrados de los gradientes.
        kwargs['sum_square'] += self.eps + np.power(kwargs['grad'], 2)
        # Calcula la tasa de aprendizaje escalada.
        lr_scaled = self.lr / np.sqrt(kwargs['sum_square'])
        # Utiliza la tasa de aprendizaje escalada para actualizar los parámetros.
        kwargs['param'] -= lr_scaled * kwargs['grad']


#### Funciones de pérdida

In [ ]:
import numpy as np
def assert_same_shape(output: np.ndarray, output_grad: np.ndarray):
    assert output.shape == output_grad.shape, \
        '''
        Dos tensores deben tener la misma forma;
        en cambio, la forma del primer tensor es {0}
        y la forma del segundo tensor es {1}.
        '''.format(tuple(output_grad.shape), tuple(output.shape))
    return None



In [ ]:
class Loss(object):
    # Constructor de la clase base para las pérdidas.
    def __init__(self):
        pass

    # Método forward para calcular la pérdida entre las predicciones y los objetivos.
    def forward(self,
                prediction: ndarray,
                target: ndarray) -> float:
        # Asegura que las predicciones y los objetivos tengan la misma forma.
        assert_same_shape(prediction, target)

        self.prediction = prediction  # Almacena las predicciones.
        self.target = target  # Almacena los objetivos.

        self.output = self._output()  # Calcula la pérdida utilizando una función interna.

        return self.output  # Devuelve el valor de la pérdida.
    
    # Método backward para calcular el gradiente de la pérdida respecto a las entradas.
    def backward(self) -> ndarray:
        self.input_grad = self._input_grad()  # Calcula el gradiente utilizando una función interna.

        # Asegura que las predicciones y el gradiente tengan la misma forma.
        assert_same_shape(self.prediction, self.input_grad)

        return self.input_grad  # Devuelve el gradiente de la entrada.

    # Método abstracto para calcular la pérdida; debe ser implementado por subclases.
    def _output(self) -> float:
        raise NotImplementedError()

    # Método abstracto para calcular el gradiente de la entrada; debe ser implementado por subclases.
    def _input_grad(self) -> ndarray:
        raise NotImplementedError()


class SoftmaxCrossEntropy(Loss):
    # Constructor de la clase para la pérdida de entropía cruzada con softmax.
    def __init__(self, eps: float=1e-9) -> None:
        super().__init__()
        self.eps = eps  # Un pequeño número para evitar inestabilidad numérica.

    # Método para calcular la salida de la pérdida de entropía cruzada con softmax.
    def _output(self) -> float:
        out = []
        # Aplica softmax a cada fila de la predicción.
        for row in self.prediction:
            out.append(softmax(row, axis=1))
        softmax_preds = np.stack(out)

        # Recorta la salida de softmax para prevenir inestabilidad numérica.
        self.softmax_preds = np.clip(softmax_preds, self.eps, 1 - self.eps)

        # Calcula la pérdida real de entropía cruzada.
        softmax_cross_entropy_loss = -1.0 * self.target * np.log(self.softmax_preds) - \
            (1.0 - self.target) * np.log(1 - self.softmax_preds)

        return np.sum(softmax_cross_entropy_loss)  # Retorna la suma total de la pérdida.

    # Método para calcular el gradiente de la entrada basado en la salida de softmax.
    def _input_grad(self) -> np.ndarray:
        return self.softmax_preds - self.target  # Gradiente de la pérdida respecto a la entrada.


 ## Redes neuronales recurrentes
Una red neuronal recurrente (RNN) es una clase de redes neuronales diseñadas para manejar secuencias de datos, como series temporales o secuencias lingüísticas. 

A diferencia de las redes neuronales tradicionales que asumen independencia entre las entradas, las RNNs tienen "memoria" sobre entradas anteriores. Esto les permite retener información a través del tiempo y utilizar esta información para influir en la salida actual, lo cual es crucial para tareas donde el contexto y el orden de los datos son importantes, como el procesamiento del lenguaje natural o el análisis de series temporales.

#### Clase RNNNode
La clase RNNNode define un nodo dentro de una red neuronal recurrente (RNN), capaz de realizar cálculos hacia adelante y hacia atrás. El método forward calcula la salida y el nuevo estado oculto del nodo basado en la entrada actual y el estado oculto anterior, utilizando matrices de pesos y sesgos. 

El método backward se encarga de la retropropagación del error, calculando los gradientes respecto a las entradas y actualizando los parámetros del modelo basados en estos gradientes. Esto permite entrenar la red para ajustar sus pesos y mejorar la precisión de sus predicciones a lo largo del tiempo.

In [ ]:
class RNNNode(object):
    # Constructor del nodo RNN, inicialmente no realiza ninguna operación específica.
    def __init__(self):
        pass

    # Método forward para calcular la salida del nodo RNN a partir de la entrada y el estado oculto anterior.
    def forward(self,
                x_in: ndarray, 
                H_in: ndarray,
                params_dict: Dict[str, Dict[str, ndarray]]
                ) -> Tuple[ndarray]:
        '''
        Calcula la salida del nodo RNN y el nuevo estado oculto.
        Args:
        x_in: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        H_in: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        Retorna x_out: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        Retorna H_out: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        '''
        self.X_in = x_in
        self.H_in = H_in
    
        # Concatena la entrada x con el estado oculto anterior H.
        self.Z = np.column_stack((x_in, H_in))
        
        # Calcula el nuevo estado oculto intermedio usando los pesos y biases.
        self.H_int = np.dot(self.Z, params_dict['W_f']['value']) + params_dict['B_f']['value']
        
        # Aplica la función de activación tanh al estado oculto intermedio.
        self.H_out = tanh(self.H_int)

        # Calcula la salida del nodo RNN.
        self.X_out = np.dot(self.H_out, params_dict['W_v']['value']) + params_dict['B_v']['value']
        
        return self.X_out, self.H_out

    # Método backward para la propagación hacia atrás del error a través del nodo RNN.
    def backward(self, 
                 X_out_grad: ndarray, 
                 H_out_grad: ndarray,
                 params_dict: Dict[str, Dict[str, ndarray]]) -> Tuple[ndarray]:
        '''
        Retropropaga el gradiente a través del nodo RNN.
        Args:
        X_out_grad: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        H_out_grad: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        Retorna X_in_grad: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        Retorna H_in_grad: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        '''
        
        # Verifica que los gradientes y las salidas tengan la misma forma.
        assert_same_shape(X_out_grad, self.X_out)
        assert_same_shape(H_out_grad, self.H_out)

        # Calcula los gradientes para los parámetros de salida y acumula en los derivados.
        params_dict['B_v']['deriv'] += X_out_grad.sum(axis=0)
        params_dict['W_v']['deriv'] += np.dot(self.H_out.T, X_out_grad)
        
        # Propaga el gradiente hacia atrás a través de la red.
        dh = np.dot(X_out_grad, params_dict['W_v']['value'].T)
        dh += H_out_grad
        
        # Calcula el gradiente del estado oculto intermedio.
        dH_int = dh * dtanh(self.H_int)
        
        # Acumula gradientes en los parámetros del estado oculto.
        params_dict['B_f']['deriv'] += dH_int.sum(axis=0)
        params_dict['W_f']['deriv'] += np.dot(self.Z.T, dH_int)     
        
        # Calcula los gradientes de entrada.
        dz = np.dot(dH_int, params_dict['W_f']['value'].T)
        X_in_grad = dz[:, :self.X_in.shape[1]]
        H_in_grad = dz[:, self.X_in.shape[1]:]
        
        return X_in_grad, H_in_grad


#### RNNLayer

La clase RNNLayer es una capa de red neuronal recurrente que maneja la propagación hacia adelante y hacia atrás de los datos a través de una secuencia de tiempo. En el método forward, la capa procesa secuencialmente la entrada utilizando nodos RNN internos, cada uno correspondiente a un paso de tiempo, manteniendo un estado oculto que pasa de un nodo a otro. Esta capa es capaz de ajustar sus pesos y sesgos para mejorar la predicción del siguiente carácter en una secuencia.

Durante la retropropagación, calcula los gradientes para actualizar los parámetros con el fin de minimizar el error en las predicciones. Esta estructura es fundamental para tareas de procesamiento de secuencias como la generación de texto, donde la dependencia temporal entre los datos es crucial.

In [ ]:
class RNNLayer(object):
    # Constructor de la clase de la capa RNN.
    # Args:
    #   hidden_size: int - Número de neuronas ocultas en la capa RNN.
    #   output_size: int - Número de caracteres en el vocabulario para predecir el siguiente carácter.
    #   weight_scale: float - Escala para la inicialización de los pesos.
    def __init__(self,
                 hidden_size: int,
                 output_size: int,
                 weight_scale: float = None):
        self.hidden_size = hidden_size  # Almacena el tamaño del estado oculto.
        self.output_size = output_size  # Almacena el tamaño de salida.
        self.weight_scale = weight_scale  # Escala de inicialización de los pesos.
        self.start_H = np.zeros((1, hidden_size))  # Estado oculto inicial.
        self.first = True  # Bandera para inicialización en el primer paso forward.

    # Método para inicializar los parámetros de la capa.
    def _init_params(self, input_: ndarray):
        self.vocab_size = input_.shape[2]  # Tamaño del vocabulario a partir de la entrada.
        # Establece la escala de peso si no se proporcionó.
        if not self.weight_scale:
            self.weight_scale = 2 / (self.vocab_size + self.output_size)
        
        self.params = {'W_f': {}, 'B_f': {}, 'W_v': {}, 'B_v': {}}
        # Inicializa pesos y sesgos con valores aleatorios normalizados.
        self.params['W_f']['value'] = np.random.normal(0.0, self.weight_scale,
                                                       (self.hidden_size + self.vocab_size, self.hidden_size))
        self.params['B_f']['value'] = np.random.normal(0.0, self.weight_scale, (1, self.hidden_size))
        self.params['W_v']['value'] = np.random.normal(0.0, self.weight_scale,
                                                       (self.hidden_size, self.output_size))
        self.params['B_v']['value'] = np.random.normal(0.0, self.weight_scale, (1, self.output_size))

        self.params['W_f']['deriv'] = np.zeros_like(self.params['W_f']['value'])
        self.params['B_f']['deriv'] = np.zeros_like(self.params['B_f']['value'])
        self.params['W_v']['deriv'] = np.zeros_like(self.params['W_v']['value'])
        self.params['B_v']['deriv'] = np.zeros_like(self.params['B_v']['value'])
        
        self.cells = [RNNNode() for _ in range(input_.shape[1])]  # Inicializa nodos RNN por cada paso de secuencia.

    # Limpia los gradientes acumulados en los parámetros.
    def _clear_gradients(self):
        for key in self.params.keys():
            self.params[key]['deriv'] = np.zeros_like(self.params[key]['deriv'])

    # Procesa la entrada a través de la capa RNN y calcula la salida para cada paso de tiempo.
    def forward(self, x_seq_in: ndarray):
        if self.first:
            self._init_params(x_seq_in)  # Inicializa parámetros en el primer paso.
            self.first = False
        
        batch_size = x_seq_in.shape[0]
        H_in = np.repeat(self.start_H, batch_size, axis=0)
        sequence_length = x_seq_in.shape[1]
        x_seq_out = np.zeros((batch_size, sequence_length, self.output_size))
        
        for t in range(sequence_length):
            x_in = x_seq_in[:, t, :]
            y_out, H_in = self.cells[t].forward(x_in, H_in, self.params)
            x_seq_out[:, t, :] = y_out
    
        self.start_H = H_in.mean(axis=0, keepdims=True)  # Actualiza el estado oculto inicial para la próxima ejecución.
        
        return x_seq_out

    # Retropropaga el error desde la salida hacia las entradas.
    def backward(self, x_seq_out_grad: ndarray):
        batch_size = x_seq_out_grad.shape[0]
        h_in_grad = np.zeros((batch_size, self.hidden_size))
        sequence_length = x_seq_out_grad.shape[1]
        x_seq_in_grad = np.zeros((batch_size, sequence_length, self.vocab_size))
        
        for t in reversed(range(sequence_length)):
            x_out_grad = x_seq_out_grad[:, t, :]
            grad_out, h_in_grad = self.cells[t].backward(x_out_grad, h_in_grad, self.params)
            x_seq_in_grad[:, t, :] = grad_out
        
        return x_seq_in_grad


#### RNNModel

El código define un modelo de red neuronal recurrente (RNN) que es capaz de procesar secuencias de datos, como series temporales o texto. El modelo está compuesto por varias capas (RNNLayer), cada una procesando la entrada y pasándola a la siguiente. El proceso de entrenamiento ocurre en pasos, donde cada paso involucra:

* Paso hacia adelante (forward): Cada entrada de la secuencia es procesada por todas las capas de la red, pasando de una a otra. Esta operación se utiliza para obtener la salida de la red que luego se compara con el objetivo real para calcular la pérdida.
* Cálculo de la pérdida: Se usa un objeto de pérdida para evaluar qué tan bien la salida de la red coincide con los objetivos esperados.

* Paso hacia atrás (backward): Una vez calculada la pérdida, se calcula el gradiente de la pérdida respecto a las salidas, y este gradiente se propaga hacia atrás a través de la red para actualizar los pesos de las neuronas en cada capa, lo que permite que la red aprenda.

* Actualización de parámetros: Basándose en los gradientes obtenidos de la retropropagación, se actualizan los parámetros de la red.

In [ ]:
class RNNModel(object):
    '''
    Clase Modelo que recibe entradas y objetivos, entrena la red y calcula la pérdida.
    '''
    def __init__(self, 
                 layers: List[RNNLayer],
                 sequence_length: int, 
                 vocab_size: int, 
                 loss: Loss):
        '''
        Inicializa el modelo de red neuronal recurrente.
        Args:
        layers: Lista de capas RNN en la red.
        sequence_length: Longitud de la secuencia que pasa a través de la red.
        vocab_size: Número de caracteres en el vocabulario.
        loss: Objeto de pérdida utilizado para calcular la pérdida durante el entrenamiento.
        '''
        self.layers = layers  # Lista de capas RNN.
        self.vocab_size = vocab_size  # Tamaño del vocabulario.
        self.sequence_length = sequence_length  # Longitud de la secuencia.
        self.loss = loss  # Objeto de pérdida.
        # Establece la longitud de la secuencia para cada capa.
        for layer in self.layers:
            setattr(layer, 'sequence_length', sequence_length)

    def forward(self, 
                x_batch: ndarray):
        '''
        Realiza la propagación hacia adelante a través de la red.
        Args:
        x_batch: Array de entrada con forma (batch_size, sequence_length, vocab_size)
        Returns:
        x_batch_in: Array de salida de la última capa.
        '''       
        for layer in self.layers:
            x_batch = layer.forward(x_batch)  # Propaga la entrada a través de cada capa.
        return x_batch
        
    def backward(self, 
                 loss_grad: ndarray):
        '''
        Realiza la retropropagación a través de la red utilizando el gradiente de la pérdida.
        Args:
        loss_grad: Gradiente de la pérdida con forma (batch_size, sequence_length, vocab_size)
        Returns:
        loss_grad: Propaga el gradiente a través de todas las capas.
        '''
        for layer in reversed(self.layers):
            loss_grad = layer.backward(loss_grad)  # Retropropaga a través de cada capa en orden inverso.
        return loss_grad
                
    def single_step(self, 
                    x_batch: ndarray, 
                    y_batch: ndarray):
        '''
        Ejecuta un único paso de entrenamiento completo:
        1. Paso hacia adelante y aplicación de softmax.
        2. Calcula la pérdida y su gradiente.
        3. Paso hacia atrás.
        4. Actualización de parámetros.
        Args:
        x_batch: Array de entrada con forma (batch_size, sequence_length, vocab_size)
        y_batch: Array objetivo correspondiente.
        Returns:
        loss: Pérdida calculada para el lote actual.
        '''
        x_batch_out = self.forward(x_batch)  # Paso hacia adelante.
        loss = self.loss.forward(x_batch_out, y_batch)  # Calcula la pérdida.
        loss_grad = self.loss.backward()  # Calcula el gradiente de la pérdida.
        for layer in self.layers:
            layer._clear_gradients()  # Limpia los gradientes en cada capa.
        self.backward(loss_grad)  # Retropropaga el gradiente de la pérdida.
        return loss


#### RNNTrainer

Este código define una clase RNNTrainer que se utiliza para entrenar un modelo de red neuronal recurrente (RNN) para la generación de texto. Utiliza un archivo de texto como datos de entrada y realiza las siguientes tareas principales:

* Inicialización: Prepara el modelo, el optimizador y los datos necesarios para el entrenamiento, incluyendo la creación de mapeos de caracteres a índices y viceversa.

* Generación de entradas y objetivos: Crea los lotes de datos de entrada y los objetivos (targets) correspondientes que el modelo intentará predecir.

* Entrenamiento: Ejecuta el proceso de entrenamiento en varias iteraciones, donde cada iteración incluye un paso hacia adelante (forward), el cálculo de la pérdida, un paso hacia atrás (backward) para la propagación del error, y la actualización de los parámetros del modelo.

* Muestreo de salidas: Opcionalmente, genera texto basado en el modelo entrenado para visualizar cómo está aprendiendo el modelo durante el entrenamiento.

In [ ]:
class RNNTrainer:
    '''
    Clase que toma un archivo de texto y un modelo, y comienza a generar caracteres.
    '''
    def __init__(self, 
                 text_file: str, 
                 model: RNNModel,
                 optim: RNNOptimizer,
                 batch_size: int = 32):
        # Leer los datos del archivo de texto.
        self.data = open(text_file, 'r').read()
        self.model = model  # Modelo de red neuronal recurrente.
        self.chars = list(set(self.data))  # Lista de caracteres únicos en el texto.
        self.vocab_size = len(self.chars)  # Tamaño del vocabulario.
        self.char_to_idx = {ch:i for i,ch in enumerate(self.chars)}  # Diccionario de caracteres a índices.
        self.idx_to_char = {i:ch for i,ch in enumerate(self.chars)}  # Diccionario inverso de índices a caracteres.
        self.sequence_length = self.model.sequence_length  # Longitud de la secuencia usada en el modelo.
        self.batch_size = batch_size  # Tamaño del lote para el entrenamiento.
        self.optim = optim  # Optimizador para ajustar los parámetros del modelo.
        setattr(self.optim, 'model', self.model)  # Establece el modelo en el optimizador.

    def _generate_inputs_targets(self, start_pos: int):
        # Genera índices para los lotes de entradas y objetivos desde una posición inicial.
        inputs_indices = np.zeros((self.batch_size, self.sequence_length), dtype=int)
        targets_indices = np.zeros((self.batch_size, self.sequence_length), dtype=int)
        
        for i in range(self.batch_size):
            inputs_indices[i, :] = np.array([self.char_to_idx[ch] 
                            for ch in self.data[start_pos + i: start_pos + self.sequence_length  + i]])
            targets_indices[i, :] = np.array([self.char_to_idx[ch] 
                         for ch in self.data[start_pos + 1 + i: start_pos + self.sequence_length + 1 + i]])

        return inputs_indices, targets_indices

    def _generate_one_hot_array(self, indices: ndarray):
        # Convierte los índices de caracteres a una representación one-hot.
        batch = []
        for seq in indices:
            one_hot_sequence = np.zeros((self.sequence_length, self.vocab_size))
            for i in range(self.sequence_length):
                one_hot_sequence[i, seq[i]] = 1.0
            batch.append(one_hot_sequence) 
        return np.stack(batch)

    def sample_output(self, input_char: int, sample_length: int):
        # Genera una muestra de salida del modelo actual, caracter por caracter.
        indices = []
        sample_model = deepcopy(self.model)  # Hace una copia del modelo para usar en muestreo.
        for i in range(sample_length):
            input_char_batch = np.zeros((1, 1, self.vocab_size))
            input_char_batch[0, 0, input_char] = 1.0
            x_batch_out = sample_model.forward(input_char_batch)
            x_softmax = batch_softmax(x_batch_out)
            input_char = np.random.choice(range(self.vocab_size), p=x_softmax.ravel())
            indices.append(input_char)
        txt = ''.join(self.idx_to_char[idx] for idx in indices)
        return txt

    def train(self, num_iterations: int, sample_every: int=100):
        # Entrena el modelo para generar caracteres.
        plot_iter = np.zeros((0))
        plot_loss = np.zeros((0))
        num_iter = 0
        start_pos = 0
        moving_average = deque(maxlen=100)
        while num_iter < num_iterations:
            if start_pos + self.sequence_length + self.batch_size + 1 > len(self.data):
                start_pos = 0
            inputs_indices, targets_indices = self._generate_inputs_targets(start_pos)
            inputs_batch, targets_batch = \
                self._generate_one_hot_array(inputs_indices), self._generate_one_hot_array(targets_indices)
            loss = self.model.single_step(inputs_batch, targets_batch)
            self.optim.step()
            moving_average.append(loss)
            ma_loss = np.mean(moving_average)
            start_pos += self.batch_size
            plot_iter = np.append(plot_iter, [num_iter])
            plot_loss = np.append(plot_loss, [ma_loss])
            if num_iter % sample_every == 0:
                plt.plot(plot_iter, plot_loss)
                display.clear_output(wait=True)
                plt.show()
                sample_text = self.sample_output(self.char_to_idx[self.data[start_pos]], 200)
                print(sample_text)
            num_iter += 1


#### Experimento

In [ ]:
capas = [RNNLayer(hidden_size=256, output_size=62)]
mod = RNNModel(layers=capas,
               vocab_size=62, sequence_length=10,
               loss=SoftmaxCrossEntropy())
optim = SGD(lr=0.001, gradient_clipping=True)
trainer = RNNTrainer('Ejemplo.txt', mod, optim)
trainer.train(10000, sample_every=100)

### Ejercicios

#### **Ejercicio 1: Redes neuronales recurrentes y compartición de pesos**

**Tema:** Redes neuronales recurrentes como compartición de pesos a lo largo del tiempo.

**Instrucciones:**

1. **Conceptualización:**
   - Explica cómo las redes neuronales recurrentes (RNN) implementan el concepto de **compartición de pesos** y por qué es beneficioso en problemas de secuencias.

2. **Análisis del código:**
   - Revisa el código proporcionado y señala dónde se implementa la compartición de pesos en la RNN.
   - Describe cómo esta implementación afecta al entrenamiento y a la capacidad del modelo para generalizar en secuencias de diferentes longitudes.

**Sugerencias:**

1. **Conceptualización:**

   Las RNNs implementan la compartición de pesos al utilizar los mismos pesos y biases en cada paso de tiempo de la secuencia. Esto significa que la transformación aplicada a la entrada en cada paso es la misma, independientemente de la posición en la secuencia.

   **Beneficios:**
   - **Reducción de parámetros:** Al compartir pesos, el número total de parámetros que el modelo necesita aprender se reduce significativamente.
   - **Generalización en secuencias variables:** Permite que el modelo generalice mejor en secuencias de diferentes longitudes, ya que aplica la misma transformación en cada paso.
   - **Captura de dependencias temporales:** Facilita el aprendizaje de patrones temporales y dependencias a largo plazo en los datos secuenciales.

2. **Análisis del código:**

   - **Implementación de compartición de pesos:**
     - En el código, la clase `RNNNode` representa un nodo de la RNN que procesa un paso de tiempo en la secuencia.
     - Los parámetros `W_f`, `B_f`, `W_v`, y `B_v` se almacenan en `params_dict` y son compartidos por todos los nodos en la secuencia.
     - Al instanciar los nodos en `self.cells = [RNNNode() for _ in range(input_.shape[1])]`, se crean múltiples nodos que utilizarán los mismos parámetros durante el forward y backward.

   - **Efecto en entrenamiento y generalización:**
     - **Entrenamiento:** La compartición de pesos permite que el gradiente acumulado durante el backward pase a través de todos los pasos de tiempo, actualizando los mismos parámetros. Esto mejora la eficiencia del entrenamiento.
     - **Generalización:** Al aplicar la misma transformación en cada paso, el modelo puede generalizar su aprendizaje a secuencias más largas o más cortas que las vistas durante el entrenamiento, siempre y cuando los patrones temporales sean consistentes.

---

#### **Ejercicio 2: Implementación de RNNs en PyTorch**

**Tema:** Implementación práctica de una RNN simple en PyTorch para un problema de clasificación de secuencias.

**Instrucciones:**

1. **Problema a resolver:**
   - Crea un modelo RNN en PyTorch que clasifique secuencias de números enteros en dos clases: secuencias cuya suma es par y secuencias cuya suma es impar.

2. **Implementación:**
   - Diseña la red neuronal con las siguientes especificaciones:
     - Una capa de embedding para mapear los números enteros a vectores de dimensión fija.
     - Una capa RNN (puedes usar `nn.RNN` o `nn.LSTM`).
     - Una capa fully connected para la clasificación final.
   - Utiliza la salida del **último paso de tiempo** para hacer la predicción.

3. **Entrenamiento:**
   - Genera un conjunto de datos sintético con secuencias de longitud fija.
   - Entrena el modelo y reporta la precisión en un conjunto de prueba.

**Sugerencia:**

1. **Problema a resolver:**

   Queremos clasificar secuencias de números enteros en dos clases:
   - **Clase 0:** Si la suma de los elementos de la secuencia es par.
   - **Clase 1:** Si la suma es impar.

2. **Implementación:**

   ```python
   import torch
   import torch.nn as nn
   import torch.optim as optim
   from torch.utils.data import Dataset, DataLoader

   # Definición del modelo
   class ParityRNN(nn.Module):
       def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
           super(ParityRNN, self).__init__()
           self.embedding = nn.Embedding(vocab_size, embedding_dim)
           self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
           self.fc = nn.Linear(hidden_dim, output_dim)
       
       def forward(self, x):
           embedded = self.embedding(x)
           output, hidden = self.rnn(embedded)
           # Usamos la salida del último paso de tiempo
           out = self.fc(hidden.squeeze(0))
           return out

   # Hiperparámetros
   VOCAB_SIZE = 10  # Números del 0 al 9
   EMBEDDING_DIM = 16
   HIDDEN_DIM = 32
   OUTPUT_DIM = 2  # Par o impar
   SEQ_LENGTH = 5
   BATCH_SIZE = 32
   NUM_EPOCHS = 5

   # Generación del conjunto de datos sintético
   class ParityDataset(Dataset):
       def __init__(self, num_samples):
           self.data = []
           self.labels = []
           for _ in range(num_samples):
               seq = torch.randint(0, VOCAB_SIZE, (SEQ_LENGTH,))
               label = seq.sum().item() % 2  # 0 si es par, 1 si es impar
               self.data.append(seq)
               self.labels.append(label)
       
       def __len__(self):
           return len(self.data)
       
       def __getitem__(self, idx):
           return self.data[idx], self.labels[idx]

   train_dataset = ParityDataset(1000)
   test_dataset = ParityDataset(200)
   train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
   test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

   # Instanciación del modelo, criterio y optimizador
   model = ParityRNN(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.Adam(model.parameters())

   # Entrenamiento
   for epoch in range(NUM_EPOCHS):
       model.train()
       total_loss = 0
       for sequences, labels in train_loader:
           optimizer.zero_grad()
           outputs = model(sequences)
           loss = criterion(outputs, labels)
           loss.backward()
           optimizer.step()
           total_loss += loss.item()
       print(f'Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {total_loss/len(train_loader)}')

   # Evaluación
   model.eval()
   correct = 0
   total = 0
   with torch.no_grad():
       for sequences, labels in test_loader:
           outputs = model(sequences)
           _, predicted = torch.max(outputs.data, 1)
           total += labels.size(0)
           correct += (predicted == labels).sum().item()
   print(f'Accuracy on test set: {100 * correct / total}%')
   ```

3. **Resultados:**

   Después del entrenamiento, el modelo debería lograr una precisión razonablemente alta en el conjunto de prueba, demostrando su capacidad para aprender la tarea de clasificación basada en la suma de la secuencia.

---

#### **Ejercicio 3: Uso de capas de embedding**

**Tema:** Implementación y beneficios de las capas de embedding en modelos de RNN.

**Instrucciones:**

1. **Conceptualización:**
   - Explica qué es una capa de embedding y por qué es útil en el procesamiento de secuencias discretas como texto o números enteros.

2. **Implementación práctica:**
   - En el modelo del **Ejercicio 2**, reemplaza la capa de embedding por una representación one-hot de los números de entrada.
   - Compara el rendimiento y el tiempo de entrenamiento del modelo con y sin la capa de embedding.

**Sugerencia:**

1. **Conceptualización:**

   Una capa de embedding es una capa que convierte entradas discretas (como índices de palabras o símbolos) en vectores densos de dimensión fija. Estos vectores densos capturan relaciones semánticas entre los elementos discretos y reducen la dimensionalidad en comparación con las representaciones one-hot.

   **Beneficios:**
   - **Dimensionalidad reducida:** Las representaciones embedding son de menor dimensión que las one-hot, lo que reduce el número de parámetros y el consumo de memoria.
   - **Captura de relaciones semánticas:** Los embeddings pueden aprender relaciones entre los elementos, mejorando el rendimiento del modelo.
   - **Eficiencia computacional:** Menos operaciones y multiplicaciones de matrices más pequeñas.

2. **Implementación práctica:**

   **Reemplazo por representación One-Hot:**

   ```python
   class ParityRNNOneHot(nn.Module):
       def __init__(self, vocab_size, hidden_dim, output_dim):
           super(ParityRNNOneHot, self).__init__()
           self.rnn = nn.RNN(vocab_size, hidden_dim, batch_first=True)
           self.fc = nn.Linear(hidden_dim, output_dim)
       
       def forward(self, x):
           # Convertimos a one-hot
           batch_size = x.size(0)
           seq_length = x.size(1)
           x_one_hot = torch.zeros(batch_size, seq_length, vocab_size)
           x_one_hot.scatter_(2, x.unsqueeze(2), 1)
           output, hidden = self.rnn(x_one_hot)
           out = self.fc(hidden.squeeze(0))
           return out

   # Instanciamos y entrenamos el modelo como antes
   model_one_hot = ParityRNNOneHot(VOCAB_SIZE, HIDDEN_DIM, OUTPUT_DIM)
   # ... (entrenamiento similar al anterior)
   ```

   **Comparación:**

   - **Rendimiento:** Es posible que el modelo con representación one-hot tenga un rendimiento similar o ligeramente inferior, ya que no captura relaciones entre los números.
   - **Tiempo de entrenamiento:** El modelo con one-hot puede ser más lento debido a la mayor dimensionalidad de las entradas y las operaciones adicionales para crear las representaciones one-hot.

---
#### **Ejercicio 4: Predicciones utilizando el último paso de tiempo**

**Tema:** Uso del último estado oculto en RNNs para hacer predicciones.

**Instrucciones:**

1. **Explicación:**
   - Justifica por qué en problemas de clasificación de secuencias es común utilizar la salida del último paso de tiempo para hacer predicciones.

2. **Implementación alternativa:**
   - Modifica el modelo del **Ejercicio 2** para utilizar la salida promedio de todos los pasos de tiempo en lugar del último estado oculto.
   - Entrena el modelo con esta modificación y compara los resultados con la versión original.

**Sugerencia:**

1. **Explicación:**

   En problemas donde la predicción depende de la secuencia completa, el último estado oculto de una RNN estándar (como `nn.RNN` o `nn.LSTM`) contiene información agregada de todos los pasos anteriores. Por lo tanto, es adecuado para capturar la representación de la secuencia entera.

2. **Implementación alternativa:**

   ```python
   class ParityRNNAverage(nn.Module):
       def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
           super(ParityRNNAverage, self).__init__()
           self.embedding = nn.Embedding(vocab_size, embedding_dim)
           self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
           self.fc = nn.Linear(hidden_dim, output_dim)
       
       def forward(self, x):
           embedded = self.embedding(x)
           output, hidden = self.rnn(embedded)
           # Promediamos las salidas de todos los pasos de tiempo
           out = self.fc(output.mean(dim=1))
           return out

   # Entrenamiento similar al anterior
   model_avg = ParityRNNAverage(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
   # ... (entrenamiento y evaluación)
   ```

   **Comparación de resultados:**

   - **Rendimiento:** Dependiendo de la tarea, promediar las salidas puede mejorar o empeorar el rendimiento. En este caso, como la suma de la secuencia determina la clase, utilizar el último estado oculto (que acumula la información) suele ser más efectivo.
   - **Interpretación:** Promediar puede diluir la información importante que se captura en los últimos pasos, especialmente en secuencias donde los elementos finales son más relevantes.

---

#### **Ejercicio 5: Mejorando el tiempo de entrenamiento con packing**

**Tema:** Uso de `pad_sequence` y `pack_padded_sequence` en PyTorch para manejar secuencias de longitud variable.

**Instrucciones:**

1. **Preparación de datos:**
   - Modifica el conjunto de datos para que las secuencias tengan longitudes variables entre 3 y 7.
   - Asegúrate de que las secuencias estén correctamente etiquetadas según la suma de sus elementos.

2. **Implementación:**
   - Utiliza `pad_sequence` para crear lotes con secuencias de diferentes longitudes.
   - Emplea `pack_padded_sequence` y `pad_packed_sequence` para que la RNN procese eficientemente las secuencias.

3. **Entrenamiento y evaluación:**
   - Entrena el modelo con las secuencias de longitud variable.
   - Compara el tiempo de entrenamiento y el rendimiento con respecto al modelo que utiliza secuencias de longitud fija.

**Sugerencia:**

1. **Preparación de datos:**

   ```python
   from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

   class VariableLengthParityDataset(Dataset):
       def __init__(self, num_samples):
           self.data = []
           self.labels = []
           self.lengths = []
           for _ in range(num_samples):
               seq_length = torch.randint(3, 8, (1,)).item()
               seq = torch.randint(0, VOCAB_SIZE, (seq_length,))
               label = seq.sum().item() % 2
               self.data.append(seq)
               self.labels.append(label)
               self.lengths.append(seq_length)
       
       def __len__(self):
           return len(self.data)
       
       def __getitem__(self, idx):
           return self.data[idx], self.labels[idx], self.lengths[idx]
   ```

2. **Implementación:**

   **Función de collate para DataLoader:**

   ```python
   def collate_fn(batch):
       batch.sort(key=lambda x: x[2], reverse=True)  # Ordenar por longitud
       sequences, labels, lengths = zip(*batch)
       sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
       labels = torch.tensor(labels)
       lengths = torch.tensor(lengths)
       return sequences_padded, labels, lengths
   ```

   **Modelo modificado:**

   ```python
   class ParityRNNPacked(nn.Module):
       def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
           super(ParityRNNPacked, self).__init__()
           self.embedding = nn.Embedding(vocab_size, embedding_dim)
           self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
           self.fc = nn.Linear(hidden_dim, output_dim)
       
       def forward(self, x, lengths):
           embedded = self.embedding(x)
           packed_input = pack_padded_sequence(embedded, lengths.cpu(), batch_first=True)
           packed_output, hidden = self.rnn(packed_input)
           out = self.fc(hidden.squeeze(0))
           return out
   ```

3. **Entrenamiento y evaluación:**

   ```python
   # Dataset y DataLoader
   train_dataset_var = VariableLengthParityDataset(1000)
   train_loader_var = DataLoader(train_dataset_var, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

   # Modelo y entrenamiento
   model_packed = ParityRNNPacked(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
   # ... (entrenamiento similar al anterior, pasando 'lengths' al forward)

   # Evaluación
   # ... (ajustar el DataLoader y el bucle de evaluación)
   ```

   **Comparación:**

   - **Tiempo de entrenamiento:** Usar `pack_padded_sequence` evita cómputos innecesarios en los pasos de tiempo que corresponden al padding, mejorando la eficiencia.
   - **Rendimiento:** El modelo debería mantener o incluso mejorar su precisión, ya que ahora maneja secuencias de longitud variable de manera más adecuada.

---

#### **Ejercicio 6: Simultáneo uso de inputs empaquetados y no empaquetados**

**Tema:** Manejar capas que no soportan inputs empaquetados después de una RNN que utiliza `pack_padded_sequence`.

**Instrucciones:**

1. **Contexto:**
   - Después de la RNN, queremos agregar una capa que no soporta inputs empaquetados, como una capa de atención personalizada.

2. **Implementación:**
   - Modifica el modelo para desempaquetar la salida de la RNN antes de pasarla a la siguiente capa.
   - Asegúrate de que el modelo sigue siendo eficiente y que maneja correctamente las secuencias de longitud variable.

**Sugerencia:**

```python
class ParityRNNWithAttention(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(ParityRNNWithAttention, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.attention = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed_input = pack_padded_sequence(embedded, lengths.cpu(), batch_first=True)
        packed_output, _ = self.rnn(packed_input)
        # Desempaquetamos la salida
        output, _ = pad_packed_sequence(packed_output, batch_first=True)
        # Aplicamos atención
        attn_weights = torch.softmax(self.attention(output), dim=1)
        attn_output = torch.sum(output * attn_weights, dim=1)
        out = self.fc(attn_output)
        return out
```

**Notas:**

- **Desempaquetado necesario:** Para aplicar la capa de atención, necesitamos la salida completa de la RNN sin empaquetar.
- **Eficiencia mantener:** Aunque desempaquetamos la salida, el uso de `pack_padded_sequence` en la RNN sigue aportando beneficios en términos de eficiencia computacional.
- **Manejo de longitudes:** Al desempaquetar, obtenemos tensores donde las secuencias más cortas tienen padding al final, lo cual manejamos correctamente en la capa de atención.


In [ ]:
## Tus respuestas

### Ejercicios adicionales

#### Ejercicio 1: Extensión a LSTM y GRU

1. Implementa LSTMNode y GRUNode: Basándote en la estructura de RNNNode, implementa dos nuevas clases, LSTMNode y GRUNode, que representen las operaciones específicas de las celdas LSTM y GRU, respectivamente.
2. Actualiza RNNLayer: Modifica la clase RNNLayer para que pueda utilizar RNNNode, LSTMNode, o GRUNode según un parámetro de configuración. Esto podría implicar agregar un argumento adicional en el constructor que especifique el tipo de nodo a utilizar.

3. Experimentación: Entrena modelos utilizando las diferentes configuraciones de nodos (RNN simple, LSTM, GRU) en un conjunto de datos de texto para comparar su rendimiento en términos de velocidad de convergencia y capacidad de generación de texto.


In [ ]:
## Tu respuesta

#### Ejercicio 2: Análisis de sentimientos usando RNN


1. Preprocesa un conjunto de datos de reseñas de películas o tweets para convertir el texto en secuencias de índices.

2. Modifica el  RNNModel:Asegúrate de que RNNModel pueda manejar tareas de clasificación agregando una capa densa al final con una activación de softmax o sigmoide.

3. Entrena y compara: Entrena el modelo usando RNN, LSTM, y GRU, y compara su efectividad en la clasificación de sentimientos.


In [ ]:
## Tu respuesta

#### Ejercicio 3: Autoencoders recurrentes para la detección de anomalías

1. Modifica la arquitectura actual para crear un autoencoder, donde la capa RNNLayer sirva como encoder y decoder. La entrada al decoder puede ser la representación codificada de la entrada más una secuencia de "start tokens" para la reconstrucción.

2. Detección de anomalías: Entrena el autoencoder en datos normales y luego utiliza el error de reconstrucción para detectar anomalías en nuevos datos.

3. Experimentación: Utiliza un conjunto de datos como el de series temporales de sensores o datos financieros para entrenar y evaluar el modelo.


In [ ]:
## Tu respuesta

#### Ejercicio 4: Mejora y optimización del RNNTrainer

1. Implementa métodos en RNNTrainer para guardar el estado del modelo en puntos específicos durante el entrenamiento y cargar modelos previamente entrenados.

2. Early stopping: Añade una comprobación de early stopping para terminar el entrenamiento si el modelo no mejora después de un número determinado de épocas.

3. Integra una programación de la tasa de aprendizaje que ajuste automáticamente el lr del optimizador basándose en el progreso del entrenamiento.

In [ ]:
## Tu respuesta